For this task we tried the NLTK and SpaCy tokenizers as well.


In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import pandas as pd
from nltk.metrics import jaccard_distance

In [35]:
df = pd.read_csv('/content/drive/MyDrive/IHLT/Exercises/STS.input.SMTeuroparl.txt',sep='\t',header=None)

In [36]:
#We convert everything to lowercase
df[0]=df.apply(lambda x: x[0].lower(), axis = 1)
df[1]=df.apply(lambda x: x[1].lower(), axis = 1)

In the exercise for Lab 2 we already checked that by turning every word to lowercase the pearson coefficient improved in every approach.

In [37]:
df['gs'] = pd.read_csv('/content/drive/MyDrive/IHLT/Exercises/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

We started with tokenizing in both approaches, since it is not possible to lemmatize after splitting the sentences by the whitespaces for example.

# With NLTK



*   when we make decisions, make reasons for them (with results if possible)











In [38]:
import nltk
nltk.download('averaged_perceptron_tagger', quiet = True)
nltk.download('wordnet', quiet = True)
nltk.download('omw-1.4', quiet = True)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
wnl = nltk.stem.WordNetLemmatizer()

## Non_Lemmatized: With tokenized version of the words


Here we present the previous result we got in Lab 2 (with tokenizing only):

In [40]:
df['doc_0_nltk'] = df.apply(lambda x: [token for token in nltk.word_tokenize(x[0])], axis = 1)
df['doc_1_nltk'] = df.apply(lambda x: [token for token in nltk.word_tokenize(x[1])], axis = 1)

In [41]:
df['jaccard_nltk'] = df.apply(lambda x: 1-jaccard_distance(set(x['doc_0_nltk']), set(x['doc_1_nltk'])), axis = 1)

In [42]:
from scipy.stats import pearsonr
pearsonr(df['gs'], df['jaccard_nltk'])[0]

0.46249513975914963

## Lemmatized: With tokenized version of the words


Let's try to lemmatize the words using NLTK:

In [43]:
df['jaccard_nltk_lemma'] = df.apply(lambda x: 1-jaccard_distance(set(map(wnl.lemmatize,x['doc_0_nltk'])), set(map(wnl.lemmatize,x['doc_1_nltk']))), axis = 1)

In [44]:
pearsonr(df['gs'], df['jaccard_nltk_lemma'])[0]

0.4707125711289244

As we expected, the results got better after lemmatizing.

# With SpaCy

Now repeating the whole process with SpaCy:

First, let's do the previous approach (Lab 2), with only tokenizing:

In [45]:
import spacy
nlp = spacy.load("en_core_web_sm")

## Non_Lemmatized: With tokenized version of the words


In [46]:
df['doc_0_spacy'] = df.apply(lambda x: [token.text for token in nlp(x[0])], axis = 1)
df['doc_1_spacy'] = df.apply(lambda x: [token.text for token in nlp(x[1])], axis = 1)

In [47]:
df['jaccard_spacy'] = df.apply(lambda x: 1-jaccard_distance(set(x['doc_0_spacy']), set(x['doc_1_spacy'])), axis = 1)

In [48]:
from scipy.stats import pearsonr
pearsonr(df['gs'], df['jaccard_spacy'])[0]

0.47389169840204165

## Lemmatized: With tokenized version of the words


In [49]:
df['doc_0_spacy_lemma'] = df.apply(lambda x: [token.lemma for token in nlp(x[0])], axis = 1)
df['doc_1_spacy_lemma'] = df.apply(lambda x: [token.lemma for token in nlp(x[1])], axis = 1)

In [50]:
df['jaccard_lemma_spacy'] = df.apply(lambda x: 1-jaccard_distance(set(x['doc_0_spacy_lemma']), set(x['doc_1_spacy_lemma'])), axis = 1)

In [51]:
pearsonr(df['gs'], df['jaccard_lemma_spacy'])[0]

0.48190657955388216

Lemmatizing also improved the results for pearson correlation. As Spacy was clearly better we will continue just using Spacy for the following heuristics.

## Non_Lemmatize: Filtering stopwords

The first heuristic we try is to filter out stopwords.

In [52]:
df['filtered_0'] = df.apply(lambda x: [token.text for token in nlp(x[0]) if not token.is_stop], axis = 1)
df['filtered_1'] = df.apply(lambda x: [token.text for token in nlp(x[1]) if not token.is_stop], axis = 1)

In [53]:
df['filtered_jaccard'] = df.apply(lambda x: 1-jaccard_distance(set(x['filtered_0']), set(x['filtered_1'])), axis = 1)

In [54]:
from scipy.stats import pearsonr
pearsonr(df['gs'], df['filtered_jaccard'])[0]

0.47682625439777815

## Lemmatized: Filtering stopwords

In [55]:
df['filtered_0_lemma'] = df.apply(lambda x: [token.lemma for token in nlp(x[0]) if not token.is_stop], axis = 1)
df['filtered_1_lemma'] = df.apply(lambda x: [token.lemma for token in nlp(x[1]) if not token.is_stop], axis = 1)

In [56]:
df['filtered_jaccard_lemma'] = df.apply(lambda x: 1-jaccard_distance(set(x['filtered_0_lemma']), set(x['filtered_1_lemma'])), axis = 1)

In [57]:
from scipy.stats import pearsonr
pearsonr(df['gs'], df['filtered_jaccard_lemma'])[0]

0.49671900365312177

We see great improvement after removing stopwords both in the non-lemmatized an in the lemmatized versions and (again) lemmatizing is better.

## Non-Lemmatizing: Removing punctuation marks as well

After analysing a few examples, we saw that it might help if we removed the punctuation marks as well.

In [58]:
df['filtered_punct_0'] = df.apply(lambda x: [token.text for token in nlp(x[0]) if not token.is_stop or not token.is_punct], axis = 1)
df['filtered_punct_1'] = df.apply(lambda x: [token.text for token in nlp(x[1]) if not token.is_stop or not token.is_punct], axis = 1)

In [59]:
df['filtered_punct_jaccard'] = df.apply(lambda x: 1-jaccard_distance(set(x['filtered_punct_0']), set(x['filtered_punct_1'])), axis = 1)

In [60]:
from scipy.stats import pearsonr
pearsonr(df['gs'], df['filtered_punct_jaccard'])[0]

0.47389169840204165

## Lemmatizing: Removing punctuation marks as well

In [61]:
df['filtered_punct_0_lemma'] = df.apply(lambda x: [token.lemma for token in nlp(x[0]) if not token.is_stop or not token.is_punct], axis = 1)
df['filtered_punct_1_lemma'] = df.apply(lambda x: [token.lemma for token in nlp(x[1]) if not token.is_stop or not token.is_punct], axis = 1)

In [62]:
df['filtered_punct_jaccard_lemma'] = df.apply(lambda x: 1-jaccard_distance(set(x['filtered_punct_0_lemma']), set(x['filtered_punct_1_lemma'])), axis = 1)

In [63]:
from scipy.stats import pearsonr
pearsonr(df['gs'], df['filtered_punct_jaccard_lemma'])[0]

0.48190657955388216

We notice the same behaviour as in the non-lemmatize case, the pearson coeficient decreases after removing punctuation in both cases. Apparently, this didn't help us get closer to the golden standard.

To conclude, we got the best results when we removed stopwords after using the SpaCy lemmatizer. This proves, that on average, lemmatizing brings us closer to the golden standars. There are counterexamples, however. See below.

An example where lemmatizing is worse:

Consider the similarity between the sentences:

"I was being careful" and "I am careful".

If we lemmatize them, they both become {I, be, careful}, so according to the Jaccard similarity, they are exactly the same, but they are indeed not - we would expect the golden standard to be less than 1.


In [64]:
print(set([token.lemma_ for token in nlp("I was being careful")]))
print(set([token.lemma_ for token in nlp("I am careful")]))

{'I', 'careful', 'be'}
{'I', 'careful', 'be'}


We can also have sentences where the Jaccard similarity decreases because of the lemmatization:

"I was being nice" and "I was being careful".

After lemmatizing, the Jaccard similarity decreases to 2/3 from 3/4. We are not sure if this is an improvement or not, because we would need the golden standard for these sentences.


In [65]:
print(set([token.lemma_ for token in nlp("I was being careful")]))
print(set([token.lemma_ for token in nlp("I was being nice")]))

{'I', 'careful', 'be'}
{'I', 'nice', 'be'}
